In [73]:

import requests
import numpy as np
import pandas as pd
from io import StringIO
from thefuzz import fuzz
from bs4 import BeautifulSoup


In [2]:
import fresh_data.get_datasets
import importlib
importlib.reload(fresh_data.get_datasets)
from fresh_data.get_datasets import *

In [30]:

# years go from start-(end-1)

dir = "FEC/"

full_df = pd.DataFrame()
for year in range(1990, 2022, 2):
    year_range = f"{year-1}-{year+1}" # for whatever reason, the files save as the "last" year

    FEC_filename = f"ConCand4_{year}_24m.xlsx"
    FEC_year_df = pd.read_excel(dir+FEC_filename,skiprows=4)
    FEC_year_df["year_range"] = year_range

    full_df = pd.concat([full_df, FEC_year_df],ignore_index=True)

full_df.drop(["Coverage End Date"],axis=1, inplace=True)

drop_mask = (~pd.isna(full_df["Candidate"])) & (~pd.isna(full_df["District"]))
full_df = full_df[drop_mask]

full_df["District"] = full_df["District"].astype(int)

# Alaska and Delaware have only one house seat each in the House of Congress. The FEC reports this district as "00", but the US census and our polarization data both identify it as "1". We will need to make this change to merge appropriately down the line:
single_seat_state_mask = (full_df["District"] == 00)
full_df.loc[single_seat_state_mask, "District"] = 1

# remove whitespaces
full_df = full_df.map(lambda x: x.strip() if isinstance(x, str) else x)

In [31]:
columns = {
    'year_range' : "year_range",
    'State' : "state_name",
    'District' : "district_code",
    'Candidate' : "candidate",
    'Party' : "party",
    'Incumbent/\nChallenger/Open' : "running_as",
    'Receipts' : "receipts",
    'Contributions \nfrom Individuals' : "contributions_from_individuals",
    'Contributions\nfrom PACs and\nOther Committees' : "contributions_from_pacs",
    'Contributions and\nLoans from \n the Candidate' : "contributions_and_loans_from_candidate",
    'Disbursements' : "disbursements",
    'Cash On Hand' : "cash_on_hand",
    'Debts' : "debts",  
}
full_df = full_df.rename(columns=columns)[['year_range', 'state_name', 'district_code', 'candidate', 'party', 'running_as', 'receipts',
       'contributions_from_individuals', 'contributions_from_pacs',
       'contributions_and_loans_from_candidate', 'disbursements',
       'cash_on_hand', 'debts']]

full_df

,year_range,state_name,district_code,candidate,party,running_as,receipts,contributions_from_individuals,contributions_from_pacs,contributions_and_loans_from_candidate,disbursements,cash_on_hand,debts
0,1989-1991,Alabama,1,"CALLAHAN, HERBERT L",Republican Party,Incumbent,318680.00,129153.00,169350.00,0.00,183910.00,236511.00,0.0
1,1989-1991,Alabama,2,"DICKINSON, WILLIAM LOUIS",Republican Party,Incumbent,425127.00,171618.00,218921.00,0.00,596096.00,250325.00,0.0
2,1989-1991,Alabama,2,"BAGGIANO, FAYE S",Democratic Party,Challenger,169109.00,38099.00,91752.00,20993.00,163663.00,5445.00,0.0
3,1989-1991,Alabama,3,"ADAMS, CHARLES H",Democratic Party,Open,277954.00,122590.00,19400.00,0.00,276975.00,978.00,55000.0
4,1989-1991,Alabama,3,"DIAL, GERALD OSCAR",Democratic Party,Open,111460.00,22420.00,14750.00,74290.00,111459.00,0.00,37894.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24782,2019-2021,Wisconsin,8,"STUCK, AMANDA",Democratic Party,Challenger,424558.24,390832.60,30183.44,0.00,407495.73,2164.80,0.0
24783,2019-2021,Wyoming,1,"CHENEY, ELIZABETH MRS.",Republican Party,Incumbent,3010251.47,1169995.46,1292490.00,0.00,3066534.91,153567.15,70000.0
24784,2019-2021,Wyoming,1,"BEACH, CARL",Democratic Party,Challenger,8591.85,8591.85,0.00,0.00,8591.85,0.00,0.0
24785,2019-2021,Wyoming,1,"GREY BULL, LYNNETTE",Democratic Party,Challenger,134648.22,130197.32,2800.00,0.00,132285.65,2362.57,0.0


In [32]:
for column in full_df.columns:
    display(column+":",full_df[column].unique())

'year_range:'

array(['1989-1991', '1991-1993', '1993-1995', '1995-1997', '1997-1999',
       '1999-2001', '2001-2003', '2003-2005', '2005-2007', '2007-2009',
       '2009-2011', '2011-2013', '2013-2015', '2015-2017', '2017-2019',
       '2019-2021'], dtype=object)

'state_name:'

array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District Of Columbia', 'Florida', 'Georgia', 'Guam', 'Hawaii',
       'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
       'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska',
       'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virgin Islands',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming',
       'Puerto Rico', 'Northern Mariana'], dtype=object)

'district_code:'

...

array([183910.  , 596096.  , 163663.  , ...,   8591.85, 132285.65,
         5622.99])

'cash_on_hand:'

array([236511.  , 250325.  ,   5445.  , ...,   2164.8 , 153567.15,
         2362.57])

'debts:'

array([     0.  ,  55000.  ,  37894.  , ...,   1980.33, 442992.72,
         5701.  ])

In [33]:
polarize_and_census = load_all_data()
mask = (polarize_and_census["state_name"]=="Alabama") & (polarize_and_census["year_range"]=="1989-1991")
polarize_and_census[mask].sort_values(by="district_code",ascending=True)


,congress,bioname,party_code,party_name,age,born,year_range,state_name,state_abbrev,district_code,...,poor exemptions,age 65 and over exemptions,age 65 and over poor exemptions,child exemptions,poor child exemptions,total exemptions under age 65,poor exemptions under age 65,median agi,mean agi,year_range_open_secrets
5,101,"CALLAHAN, Herbert Leon (Sonny)",200,Republican,89.0,1932.0,1989-1991,Alabama,AL,1,...,655489,218161,32235,1160677,270932,3179239,623254,18495,26893,1989-1990
0,101,"DICKINSON, William Louis",200,Republican,83.0,1925.0,1989-1991,Alabama,AL,2,...,655489,218161,32235,1160677,270932,3179239,623254,18495,26893,1989-1990
2,101,"NICHOLS, William Flynt",100,Democrat,70.0,1918.0,1989-1991,Alabama,AL,3,...,655489,218161,32235,1160677,270932,3179239,623254,18495,26893,1989-1990
7,101,"BROWDER, John Glen",100,Democrat,46.0,1943.0,1989-1991,Alabama,AL,3,...,655489,218161,32235,1160677,270932,3179239,623254,18495,26893,1989-1990
1,101,"BEVILL, Tom",100,Democrat,84.0,1921.0,1989-1991,Alabama,AL,4,...,655489,218161,32235,1160677,270932,3179239,623254,18495,26893,1989-1990
3,101,"FLIPPO, Ronnie Gene",100,Democrat,52.0,1937.0,1989-1991,Alabama,AL,5,...,655489,218161,32235,1160677,270932,3179239,623254,18495,26893,1989-1990
4,101,"ERDREICH, Ben",100,Democrat,51.0,1938.0,1989-1991,Alabama,AL,6,...,655489,218161,32235,1160677,270932,3179239,623254,18495,26893,1989-1990
6,101,"HARRIS, Claude, Jr.",100,Democrat,54.0,1940.0,1989-1991,Alabama,AL,7,...,655489,218161,32235,1160677,270932,3179239,623254,18495,26893,1989-1990


In [34]:
full_df.district_code.dtypes

dtype('int64')

In [37]:
polarize_and_census.district_code.dtypes
polarize_and_census["year_range"].unique()

array(['1989-1991', '1991-1993', '1993-1995', '1995-1997', '1997-1999',
       '1999-2001', '2001-2003', '2003-2005', '2005-2007', '2007-2009',
       '2009-2011', '2011-2013', '2013-2015', '2015-2017', '2017-2019',
       '2019-2021'], dtype=object)

In [77]:

# Takes a ~minute! Manually merging on such a large dataset is painful!
def fuzzy_entity_res(p_name, f_name):
    prediction = fuzz.partial_token_sort_ratio(p_name,f_name)
    return prediction

def check_subset(x, fec_df):
    """
        Perform entity resolution on a record in the polarize and census df 
        Only parses a subset of the FEC df which has matches in year, state, and district
    """

    # Get subset of members from current year range and next year range (campaign data are from the years prior to service)
    next_year_range = f'{int(x["year_range"][-4:])}-{int(x["year_range"][-4:])+2}'
    match_subset = ((fec_df["year_range"] == x["year_range"]) | (fec_df["year_range"] == next_year_range)) & (fec_df["district_code"] == x["district_code"]) & (fec_df["state_name"] == x["state_name"])
    fec_subset = fec_df[match_subset]

    # instantiate the columns from FEC in the merging table:
    for column in fec_subset.columns:
        x[column] = 0

    if len(fec_subset) == 0:
        print("no match in FEC found for:")
        print(x["year_range"], "|", x["district_code"], "|", x["state_name"], "|", x["bioname"])
        return x

    fec_subset["distance"] = fec_subset.apply(lambda y: fuzzy_entity_res(x["bioname"],y["candidate"]),axis=1)
    closest = fec_subset["distance"].max()
    match_record = fec_subset.loc[fec_subset["distance"]==closest].iloc[0]
    if closest < 70: # YOUNG, Donald Edwin - YOUNG, DONALD E is scored as 73, this should be a match.
        print(f"no match, closest: {closest}, for {x['bioname']} - {match_record['candidate']}")
        return x
    else: # if we have a match, replace instantiated values with those of our match:
        # print(f"match, on closest: {closest}, for {x['bioname']} - {match_record['candidate']}")
        for column in fec_subset.columns:
            x[column] = match_record[column]
        return x

def fuzzy_merge(polarize_census, fec_df):
    return polarize_census.apply(lambda x: check_subset(x, fec_df), axis=1)

# merged_df = 
result = fuzzy_merge(polarize_and_census, full_df)
result.columns

no match, closest: 44, for NICHOLS, William Flynt - RICE, JOHN
Index(['year_range', 'state_name', 'district_code', 'candidate', 'party',
       'running_as', 'receipts', 'contributions_from_individuals',
       'contributions_from_pacs', 'contributions_and_loans_from_candidate',
       'disbursements', 'cash_on_hand', 'debts', 'distance'],
      dtype='object')
no match, closest: 48, for FLIPPO, Ronnie Gene - GREER, BAYLESS LYNN
Index(['year_range', 'state_name', 'district_code', 'candidate', 'party',
       'running_as', 'receipts', 'contributions_from_individuals',
       'contributions_from_pacs', 'contributions_and_loans_from_candidate',
       'disbursements', 'cash_on_hand', 'debts', 'distance'],
      dtype='object')
no match, closest: 50, for ROBINSON, Tommy Franklin - FULLER, RON
Index(['year_range', 'state_name', 'district_code', 'candidate', 'party',
       'running_as', 'receipts', 'contributions_from_individuals',
       'contributions_from_pacs', 'contributions_and_loans_

Index(['age', 'age 65 and over exemptions', 'age 65 and over poor exemptions',
       'bioname', 'born', 'candidate', 'cash_on_hand', 'child exemptions',
       'congress', 'contributions_and_loans_from_candidate',
       'contributions_from_individuals', 'contributions_from_pacs', 'debts',
       'disbursements', 'distance', 'district_code', 'icpsr', 'mean agi',
       'median agi', 'nokken_poole_dim1', 'nokken_poole_dim2', 'nominate_dim1',
       'nominate_dim2', 'nominate_geo_mean_probability',
       'nominate_log_likelihood', 'nominate_number_of_errors',
       'nominate_number_of_votes', 'party', 'party_code', 'party_name',
       'poor child exemptions', 'poor exemptions',
       'poor exemptions under age 65', 'receipts', 'running_as',
       'state_abbrev', 'state_fips', 'state_icpsr', 'state_name',
       'total exemptions', 'total exemptions under age 65', 'year_range',
       'year_range_open_secrets'],
      dtype='object')

### Weird edge cases:

Sander Levin is a representative from Michigan who at two points in his career represented two different districts in Michigan. From 1983-1993, he represented the 17th district, and did not recieve any contributions (at least on file with the FEC). In 1993, he campaigned and won the 12th district House seat for Michigan, which in 2012 was redistricted into the 9th district. In 2017, he announced he would not run for reelection in 2018. Instead, his son, Andy Levin, became his successor as the representative for the 9th district. 

Alaska, Wyoming, Montana, North Dakota, South Dakota, Vermont, and Delaware all have 

In [78]:
result.to_csv("polar_poverty_payments.csv")